In [1]:
import torch
from PIL import Image, ImageDraw
from tqdm.auto import tqdm
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import cv2
import PyQt5


## Load Data

### Data Info
train.csv (video에 대한 Metadata)  
(gap = 공백)  
- video_id : 해당 이미지의 비디오 번호(0, 1, 2)  
- sequence : 비디오의 공백 없는 id subset [40258, 45518, 59337, 8399, 45015..]    
(일종의 순간순간의 연속적인 사건들 이 뜻?)  
- video_frame : 비디오 내에서의 이미지의 프레임 번호(공백 있을 수 있음)  
- sequence_frame : 해당 sequence에서의 프레임 번호  
- image_id : 이미지 ID(format : {video_id}_{video_frame})  
- annotations : 불가사리의 bounding box 정보(xmin, ymin, 가로길이, 세로길이)  

In [3]:
train = pd.read_csv('./data/train.csv')
train[:10]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]
5,0,40258,5,5,0-5,[]
6,0,40258,6,6,0-6,[]
7,0,40258,7,7,0-7,[]
8,0,40258,8,8,0-8,[]
9,0,40258,9,9,0-9,[]


In [4]:
train[(train.annotations.str.len() > 2) & (train.video_id == 0)]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
16,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]"
17,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]"
18,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]"
19,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]"
20,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]"
...,...,...,...,...,...,...
6674,0,996,12314,889,0-12314,"[{'x': 1028, 'y': 637, 'width': 42, 'height': ..."
6675,0,996,12315,890,0-12315,"[{'x': 1032, 'y': 650, 'width': 42, 'height': ..."
6676,0,996,12316,891,0-12316,"[{'x': 1036, 'y': 663, 'width': 42, 'height': ..."
6677,0,996,12317,892,0-12317,"[{'x': 1040, 'y': 678, 'width': 42, 'height': ..."


In [5]:
train[train.video_frame == 490]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
7198,1,60510,490,490,1-490,"[{'x': 932, 'y': 698, 'width': 32, 'height': 36}]"
15430,2,37114,490,490,2-490,[]


In [6]:
import ast
# ast.literal_eval(train.iloc[16].annotations)
temp = train.iloc[16].annotations
print(temp)
print(type(temp))

[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]
<class 'str'>


In [7]:
a = ast.literal_eval(train.iloc[16].annotations)
print(a)
print(type(a))
# ast.literal_eval()로 문자열 변수에 할당된 표현식으로부터 원래의 데이터를 얻을 수 있다.

[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]
<class 'list'>


코드 참조 : https://www.kaggle.com/diegoalejogm/great-barrier-reefs-eda-with-animations/notebook

In [8]:
# plt.show를 new window에서 볼 수 있게끔
%matplotlib qt
def fetch_image_list(df_tmp, video_id, num_images, start_frame_idx):
    def fetch_image(frame_id):
        path = './data/train_images/video_{}/{}.jpg'
        raw_image = Image.open(path.format(video_id, frame_id))
        row_frame = df_tmp[(df_tmp.video_id == video_id) & (df_tmp.video_frame == frame_id)].iloc[0]
        bounding_boxes = ast.literal_eval(row_frame.annotations)

        for box in bounding_boxes:
            draw = ImageDraw.Draw(raw_image)
            x0, y0, x1, y1 = (box['x'], box['y'], box['x'] + box['width'], box['y'] + box['height'])
            draw.rectangle((x0, y0, x1, y1), outline=180, width=3)
        return raw_image
    return [np.array(fetch_image(start_frame_idx + index)) for index in range(num_images)]

images= fetch_image_list(train, video_id=1, num_images=2, start_frame_idx=490)

# interpolation은 보간법을 뜻하며 픽셀들의 축 위치 간격을 보정하여 이미지가 자연스러운 모양으로 보일 수 있게 함.
# nearest는 가장 고해상도의 보간법
plt.figure()
plt.imshow(images[-1], interpolation='nearest')
plt.axis('off')
plt.show()
# plt.close()

In [10]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

rc('animation', html='jshtml')
def create_animation(ims):
    fig=plt.figure(figsize=(9, 9))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

create_animation(images)